# 🎭 Facial Emotion Recognition v2.0 - Colab Training

**Architecture**: EfficientNetV2-S | **Dataset**: FER-2013 | **GPU**: T4

## What This Notebook Does

1. ✅ Mounts Google Drive (models auto-save)
2. ✅ Creates project structure locally (fast)
3. ✅ Trains with SOTA techniques
4. ✅ Saves results to your Drive

**Storage**: Training runs locally (fast), models save to Drive (persistent)

## 🔧 Setup

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print('✅ Drive mounted')

In [ ]:
# Verify GPU
import torch

if torch.cuda.is_available():
    print(f'✅ GPU: {torch.cuda.get_device_name(0)}')
    print(f'Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')
else:
    print('❌ No GPU! Go to Runtime > Change runtime type > T4 GPU')
    raise RuntimeError('GPU required for training')

In [ ]:
# Install dependencies
!pip install -q timm==0.9.12 albumentations==1.3.1 onnx==1.15.0
print('✅ Dependencies installed')

## 📂 Project Setup with Drive Symlinks

In [ ]:
import os
from pathlib import Path

# Create Drive folders (persistent)
drive_base = Path('/content/drive/MyDrive/Emotion_Detection_v2')
drive_models = drive_base / 'models'
drive_results = drive_base / 'results'
drive_models.mkdir(parents=True, exist_ok=True)
drive_results.mkdir(parents=True, exist_ok=True)

# Create local project
project_dir = Path('/content/emotion_detection_v2')
project_dir.mkdir(exist_ok=True)
(project_dir / 'src' / 'models').mkdir(parents=True, exist_ok=True)
(project_dir / 'src' / 'data').mkdir(parents=True, exist_ok=True)
(project_dir / 'src' / 'utils').mkdir(parents=True, exist_ok=True)
(project_dir / 'data').mkdir(exist_ok=True)

# Create symlinks to Drive
local_models = project_dir / 'models'
local_results = project_dir / 'results'
if local_models.exists() and local_models.is_symlink():
    local_models.unlink()
if local_results.exists() and local_results.is_symlink():
    local_results.unlink()
local_models.symlink_to(drive_models)
local_results.symlink_to(drive_results)

print('✅ Project structure created')
print(f'   Models → Drive: {drive_models}')
print(f'   Results → Drive: {drive_results}')
print('\n💡 All models and results will auto-save to your Google Drive!')

%cd /content/emotion_detection_v2

## 📥 Load Dataset

**Choose ONE option below:**

In [ ]:
# Option 1: Kaggle Download (requires kaggle.json)
# Upload your kaggle.json to Drive first, then uncomment:

# !mkdir -p ~/.kaggle
# !cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# !kaggle datasets download -d msambare/fer2013 -p data/ --unzip
# print('✅ FER-2013 downloaded from Kaggle')

In [ ]:
# Option 2: Copy from your Drive (if you already have it)
# Uncomment if your data is already in Drive:

import shutil
drive_data = '/content/drive/MyDrive/Facial_Emotion_Detection/data'
if Path(drive_data).exists():
    print('Copying data from Drive...')
    shutil.copytree(drive_data, 'data', dirs_exist_ok=True)
    print('✅ Data copied from Drive')
else:
    print('❌ Data not found in Drive. Use Option 1 or check path.')

In [ ]:
# Verify data structure
!ls -lh data/
!ls data/train/ 2>/dev/null || ls data/Test/ 2>/dev/null || echo '⚠️ Check data structure'

## 📝 Create Source Code

In [ ]:
# Create package files
!touch src/__init__.py src/models/__init__.py src/data/__init__.py src/utils/__init__.py

In [ ]:
%%writefile src/models/efficientnet.py
"""EfficientNetV2 for emotion recognition"""
import torch
import torch.nn as nn
import timm

class EmotionEfficientNet(nn.Module):
    def __init__(self, model_name='tf_efficientnetv2_s', num_classes=7, pretrained=True, dropout=0.3):
        super().__init__()
        self.backbone = timm.create_model(model_name, pretrained=pretrained, num_classes=0, global_pool='avg')
        self.num_features = self.backbone.num_features
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(self.num_features, num_classes)
        )
        self._init_classifier()
    
    def _init_classifier(self):
        for m in self.classifier.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
    
    def forward(self, x):
        return self.classifier(self.backbone(x))
    
    def get_params_groups(self, lr, backbone_lr_mult=0.1):
        return [
            {'params': self.backbone.parameters(), 'lr': lr * backbone_lr_mult},
            {'params': self.classifier.parameters(), 'lr': lr}
        ]

def create_model(model_name='tf_efficientnetv2_s', num_classes=7, pretrained=True, dropout=0.3):
    return EmotionEfficientNet(model_name, num_classes, pretrained, dropout)

In [ ]:
%%writefile src/data/dataset.py
"""Dataset loader"""
from pathlib import Path
from PIL import Image
import numpy as np
from torch.utils.data import Dataset

EMOTION_LABELS = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Neutral', 5: 'Sad', 6: 'Surprise'}

class EmotionDataset(Dataset):
    def __init__(self, data_dir, split='train', transform=None):
        self.transform = transform
        self.samples = self._load(Path(data_dir) / split)
        print(f'Loaded {len(self.samples)} {split} samples')
    
    def _load(self, split_dir):
        samples = []
        for emotion_dir in sorted(split_dir.iterdir()):
            if not emotion_dir.is_dir():
                continue
            label = self._get_label(emotion_dir.name)
            for ext in ['*.jpg', '*.png']:
                for img_path in emotion_dir.glob(ext):
                    samples.append((str(img_path), label))
        return samples
    
    def _get_label(self, name):
        mapping = {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
        return mapping.get(name.lower(), 0)
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
%%writefile src/data/transforms.py
"""Data augmentation"""
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np

IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

def get_train_transforms(img_size=224):
    return A.Compose([
        A.Resize(img_size, img_size),
        A.HorizontalFlip(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.15, rotate_limit=15, p=0.5),
        A.OneOf([A.MotionBlur(p=1), A.GaussianBlur(p=1)], p=0.3),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
        A.CoarseDropout(max_holes=8, max_height=img_size//10, max_width=img_size//10, p=0.3),
        A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
        ToTensorV2()
    ])

def get_val_transforms(img_size=224):
    return A.Compose([
        A.Resize(img_size, img_size),
        A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
        ToTensorV2()
    ])

class AlbumentationsWrapper:
    def __init__(self, transform):
        self.transform = transform
    
    def __call__(self, image):
        if hasattr(image, 'convert'):
            image = np.array(image.convert('RGB'))
        return self.transform(image=image)['image']

In [ ]:
%%writefile src/utils/training_utils.py
"""Training utilities"""
import torch
import torch.nn as nn
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

class ExponentialMovingAverage:
    def __init__(self, model, decay=0.9999):
        self.decay = decay
        self.shadow = {n: p.data.clone() for n, p in model.named_parameters() if p.requires_grad}
    
    @torch.no_grad()
    def update(self, model):
        for n, p in model.named_parameters():
            if p.requires_grad:
                self.shadow[n] = self.decay * self.shadow[n] + (1 - self.decay) * p.data
    
    def apply_shadow(self, model):
        self.backup = {n: p.data.clone() for n, p in model.named_parameters() if p.requires_grad}
        for n, p in model.named_parameters():
            if p.requires_grad:
                p.data = self.shadow[n]
    
    def restore(self, model):
        for n, p in model.named_parameters():
            if p.requires_grad:
                p.data = self.backup[n]

class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, smoothing=0.1):
        super().__init__()
        self.smoothing = smoothing
    
    def forward(self, pred, target):
        n_class = pred.size(1)
        if target.dim() == 1:
            target = torch.nn.functional.one_hot(target, n_class).float()
        target = target * (1 - self.smoothing) + self.smoothing / n_class
        return -(target * torch.nn.functional.log_softmax(pred, 1)).sum(1).mean()

class AverageMeter:
    def __init__(self):
        self.reset()
    def reset(self):
        self.val = self.avg = self.sum = self.count = 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def compute_metrics(y_true, y_pred):
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'f1_macro': f1_score(y_true, y_pred, average='macro'),
        'f1_weighted': f1_score(y_true, y_pred, average='weighted')
    }

def plot_confusion_matrix(y_true, y_pred, class_names, save_path):
    cm = confusion_matrix(y_true, y_pred).astype('float')
    cm = cm / (cm.sum(axis=1)[:, np.newaxis] + 1e-6)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='.2f', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.ylabel('True'); plt.xlabel('Predicted'); plt.title('Confusion Matrix')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

## 🚀 Training

In [ ]:
import sys
sys.path.insert(0, '/content/emotion_detection_v2')

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np

from src.models.efficientnet import create_model
from src.data.dataset import EmotionDataset, EMOTION_LABELS
from src.data.transforms import get_train_transforms, get_val_transforms, AlbumentationsWrapper
from src.utils.training_utils import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'✅ Using {device}')

In [ ]:
# Configuration
CONFIG = {
    'img_size': 224,
    'batch_size': 64,  # Reduce to 32 if GPU OOM
    'epochs': 50,      # Try 30 for faster testing
    'lr': 1e-3,
    'weight_decay': 1e-4,
    'label_smoothing': 0.1,
    'ema_decay': 0.9995,
    'warmup_epochs': 5,
}

print('Configuration:', CONFIG)

In [ ]:
# Load datasets
train_dataset = EmotionDataset('data', 'train', AlbumentationsWrapper(get_train_transforms(CONFIG['img_size'])))
# Try 'test' or 'Test' depending on your folder structure
val_dataset = EmotionDataset('data', 'test', AlbumentationsWrapper(get_val_transforms(CONFIG['img_size'])))

train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True, num_workers=2, pin_memory=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=2, pin_memory=True)

print(f'Train: {len(train_dataset)}, Val: {len(val_dataset)}, Batches: {len(train_loader)}')

In [ ]:
# Create model
model = create_model().to(device)
criterion = LabelSmoothingCrossEntropy(CONFIG['label_smoothing'])
optimizer = torch.optim.AdamW(model.get_params_groups(CONFIG['lr'], 0.1), weight_decay=CONFIG['weight_decay'])

from torch.optim.lr_scheduler import CosineAnnealingLR, LinearLR, SequentialLR
warmup = LinearLR(optimizer, start_factor=0.1, total_iters=CONFIG['warmup_epochs']*len(train_loader))
cosine = CosineAnnealingLR(optimizer, T_max=(CONFIG['epochs']-CONFIG['warmup_epochs'])*len(train_loader), eta_min=1e-6)
scheduler = SequentialLR(optimizer, [warmup, cosine], [CONFIG['warmup_epochs']*len(train_loader)])

ema = ExponentialMovingAverage(model, CONFIG['ema_decay'])
scaler = torch.cuda.amp.GradScaler()

print(f'✅ Model: {sum(p.numel() for p in model.parameters())/1e6:.1f}M params')

In [ ]:
# Training functions
def train_epoch(model, loader, criterion, optimizer, scheduler, scaler, ema):
    model.train()
    loss_meter = AverageMeter()
    for images, labels in tqdm(loader, desc='Training'):
        images, labels = images.to(device), labels.to(device)
        with torch.cuda.amp.autocast():
            logits = model(images)
            loss = criterion(logits, labels)
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        ema.update(model)
        loss_meter.update(loss.item(), images.size(0))
    return loss_meter.avg

@torch.no_grad()
def validate(model, loader):
    model.eval()
    all_preds, all_labels = [], []
    for images, labels in tqdm(loader, desc='Validating', leave=False):
        images = images.to(device)
        logits = model(images)
        all_preds.extend(logits.argmax(1).cpu().numpy())
        all_labels.extend(labels.numpy())
    return compute_metrics(np.array(all_labels), np.array(all_preds)), np.array(all_preds), np.array(all_labels)

In [ ]:
# Training loop
best_f1 = 0
history = {'train_loss': [], 'val_f1': [], 'ema_val_f1': []}

for epoch in range(CONFIG['epochs']):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, scheduler, scaler, ema)
    val_metrics, _, _ = validate(model, val_loader)
    
    ema.apply_shadow(model)
    ema_metrics, _, _ = validate(model, val_loader)
    ema.restore(model)
    
    history['train_loss'].append(train_loss)
    history['val_f1'].append(val_metrics['f1_macro'])
    history['ema_val_f1'].append(ema_metrics['f1_macro'])
    
    print(f"Epoch {epoch+1}/{CONFIG['epochs']}: Loss={train_loss:.4f}, F1={val_metrics['f1_macro']:.4f}, EMA F1={ema_metrics['f1_macro']:.4f}")
    
    if ema_metrics['f1_macro'] > best_f1:
        best_f1 = ema_metrics['f1_macro']
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'ema_shadow': ema.shadow,
            'best_f1': best_f1
        }, 'models/best_model.pth')
        print(f'  ✅ Saved best model (F1={best_f1:.4f}) to Drive!')

print(f'\n✅ Training complete! Best F1: {best_f1:.4f}')

## 📊 Evaluation

In [ ]:
# Load best model
checkpoint = torch.load('models/best_model.pth')
model.load_state_dict(checkpoint['model_state_dict'])
for n, p in model.named_parameters():
    if n in checkpoint['ema_shadow']:
        p.data = checkpoint['ema_shadow'][n]

test_metrics, test_preds, test_labels = validate(model, val_loader)

print('\n' + '='*50)
print('FINAL TEST RESULTS')
print('='*50)
print(f"Accuracy: {test_metrics['accuracy']:.4f} ({test_metrics['accuracy']*100:.2f}%)")
print(f"F1 Macro: {test_metrics['f1_macro']:.4f}")
print(f"F1 Weighted: {test_metrics['f1_weighted']:.4f}")

with open('results/test_results.txt', 'w') as f:
    f.write(f"Test Accuracy: {test_metrics['accuracy']:.4f}\n")
    f.write(f"F1 Macro: {test_metrics['f1_macro']:.4f}\n")
    f.write(f"F1 Weighted: {test_metrics['f1_weighted']:.4f}\n")

print('\n✅ Results saved to Drive!')

In [ ]:
# Confusion matrix
class_names = [EMOTION_LABELS[i] for i in range(7)]
plot_confusion_matrix(test_labels, test_preds, class_names, 'results/confusion_matrix.png')

from IPython.display import Image, display
display(Image('results/confusion_matrix.png'))
print('✅ Confusion matrix saved!')

In [ ]:
# Training curves
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(14, 5))
ax[0].plot(history['train_loss'])
ax[0].set_title('Training Loss')
ax[0].set_xlabel('Epoch')
ax[1].plot(history['val_f1'], label='Val')
ax[1].plot(history['ema_val_f1'], label='EMA Val')
ax[1].set_title('F1 Score')
ax[1].set_xlabel('Epoch')
ax[1].legend()
plt.savefig('results/training_curves.png', dpi=300, bbox_inches='tight')
plt.show()
print('✅ Training curves saved!')

## 💾 Export to ONNX

In [ ]:
# Export to ONNX
model.eval()
dummy_input = torch.randn(1, 3, 224, 224).to(device)

torch.onnx.export(
    model, dummy_input, 'models/emotion_model.onnx',
    input_names=['input'], output_names=['output'],
    dynamic_axes={'input': {0: 'batch'}, 'output': {0: 'batch'}}
)

print('✅ ONNX model exported!')
print('\n📂 Files in your Drive:')
!ls -lh /content/drive/MyDrive/Emotion_Detection_v2/models/
!ls -lh /content/drive/MyDrive/Emotion_Detection_v2/results/

## ✅ Complete!

**Saved to Google Drive** (`/Emotion_Detection_v2/`):
- ✅ `models/best_model.pth` - Best checkpoint
- ✅ `models/emotion_model.onnx` - ONNX export
- ✅ `results/confusion_matrix.png`
- ✅ `results/training_curves.png`
- ✅ `results/test_results.txt`

Download these from your Drive for local use!